# HW4 - Deep Learning Course
## Nima Samadi
## Problem 1

In [1]:
%mkdir /content/HW4
%cd /content/HW4
!wget 'https://drive.google.com/uc?export=download&id=11odow8zpwct8GpTYx_-qx-oW56ch3a8Q&confirm=T' -O ferdousi.txt
!ls

/content/HW4
--2023-02-05 16:44:37--  https://drive.google.com/uc?export=download&id=11odow8zpwct8GpTYx_-qx-oW56ch3a8Q&confirm=T
Resolving drive.google.com (drive.google.com)... 172.217.194.102, 172.217.194.113, 172.217.194.100, ...
Connecting to drive.google.com (drive.google.com)|172.217.194.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-10-a8-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/7dmas189i8udtejf7bh6vc9hnqimornv/1675615425000/04834621180226443203/*/11odow8zpwct8GpTYx_-qx-oW56ch3a8Q?e=download&uuid=656ba2e5-c5bf-4830-92aa-2ff62f24fe5d [following]
--2023-02-05 16:44:38--  https://doc-10-a8-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/7dmas189i8udtejf7bh6vc9hnqimornv/1675615425000/04834621180226443203/*/11odow8zpwct8GpTYx_-qx-oW56ch3a8Q?e=download&uuid=656ba2e5-c5bf-4830-92aa-2ff62f24fe5d
Resolving doc-10-a8-docs.googleusercontent.com (doc-10-a8-docs.googleusercontent.co

In [2]:
!pip install -q transformers
!pip install -q hazm
!pip install -q clean-text[gpl]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 69.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.7/316.7 KB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 KB 29.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 80.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 KB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 KB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 KB 8.5 MB/s eta 0:00:00


## Dataset loading and preperation:

In [3]:
%cd /content/HW4

/content/HW4


In [4]:
import pandas as pd
import os
import numpy as np
import hazm
from tqdm.notebook import tqdm
import plotly.express as px
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
import plotly.graph_objects as go
import collections

In [5]:
data = pd.read_csv("ferdousi.txt", sep="\n")
data.drop(0, axis=0, inplace=True)
data.drop(data.shape[0], axis=0, inplace=True)
data.rename(columns={data.columns[0]: 'poem'}, inplace=True)
data.reset_index(drop=True)
data

,poem
1,به نام خداوند جان و خرد
2,کزین برتر اندیشه برنگذرد
3,خداوند نام و خداوند جای
4,خداوند روزی ده رهنمای
5,خداوند کیوان و گردان سپهر
...,...
99212,به نام جهانداور کردگار
99213,چواین نامور نامه آمد ببن
99214,ز من روی کشور شود پرسخن
99215,از آن پس نمیرم که من زنده ام


In [6]:
# 1: mesra1
# 2: mesra2
mesra = np.zeros(data.shape[0], dtype=int)
indices = np.array(range(data.shape[0]))
mesra[indices % 2 == 0] = 1
mesra[indices % 2 == 1] = 2
data = data.assign(mesra=mesra)
data

,poem,mesra
1,به نام خداوند جان و خرد,1
2,کزین برتر اندیشه برنگذرد,2
3,خداوند نام و خداوند جای,1
4,خداوند روزی ده رهنمای,2
5,خداوند کیوان و گردان سپهر,1
...,...,...
99212,به نام جهانداور کردگار,2
99213,چواین نامور نامه آمد ببن,1
99214,ز من روی کشور شود پرسخن,2
99215,از آن پس نمیرم که من زنده ام,1


In [7]:
mesra1 = data[data['mesra'] == 1]['poem'].to_numpy()
mesra2 = data[data['mesra'] == 2]['poem'].to_numpy()
total_data = mesra1 + ' ' + mesra2
total_data = np.stack([total_data, total_data], axis=1)
total_data[:-1, 1] = total_data[1:, 1]
total_data[-1, 1] = total_data[0, 1]

In [8]:
total_data.shape

(49608, 2)

In [9]:
total_data_df = pd.DataFrame({'input': total_data[:, 0], 'output': total_data[:, 1]})
total_data_df

,input,output
0,به نام خداوند جان و خرد کزین برتر اندیشه برنگذرد,خداوند نام و خداوند جای خداوند روزی ده رهنمای
1,خداوند نام و خداوند جای خداوند روزی ده رهنمای,خداوند کیوان و گردان سپهر فروزنده ماه و ناهید ...
2,خداوند کیوان و گردان سپهر فروزنده ماه و ناهید ...,ز نام و نشان و گمان برترست نگارندهٔ بر شده پیکرست
3,ز نام و نشان و گمان برترست نگارندهٔ بر شده پیکرست,به بینندگان آفریننده را نبینی مرنجان دو بیننده را
4,به بینندگان آفریننده را نبینی مرنجان دو بیننده را,نیابد بدو نیز اندیشه راه که او برتر از نام و ا...
...,...,...
49603,همش رای و هم دانش وهم نسب چراغ عجم آفتاب عرب,سرآمد کنون قصهٔ یزدگرد به ماه سفندار مد روز ارد
49604,سرآمد کنون قصهٔ یزدگرد به ماه سفندار مد روز ارد,ز هجرت شده پنج هشتادبار به نام جهانداور کردگار
49605,ز هجرت شده پنج هشتادبار به نام جهانداور کردگار,چواین نامور نامه آمد ببن ز من روی کشور شود پرسخن
49606,چواین نامور نامه آمد ببن ز من روی کشور شود پرسخن,از آن پس نمیرم که من زنده ام که تخم سخن من پرا...


In [10]:
total_data_df['input_poem_len'] = total_data_df['input'].apply(lambda t: len(hazm.word_tokenize(t)))
total_data_df['output_poem_len'] = total_data_df['output'].apply(lambda t: len(hazm.word_tokenize(t)))
total_data_df

,input,output,input_poem_len,output_poem_len
0,به نام خداوند جان و خرد کزین برتر اندیشه برنگذرد,خداوند نام و خداوند جای خداوند روزی ده رهنمای,10,9
1,خداوند نام و خداوند جای خداوند روزی ده رهنمای,خداوند کیوان و گردان سپهر فروزنده ماه و ناهید ...,9,11
2,خداوند کیوان و گردان سپهر فروزنده ماه و ناهید ...,ز نام و نشان و گمان برترست نگارندهٔ بر شده پیکرست,11,11
3,ز نام و نشان و گمان برترست نگارندهٔ بر شده پیکرست,به بینندگان آفریننده را نبینی مرنجان دو بیننده را,11,9
4,به بینندگان آفریننده را نبینی مرنجان دو بیننده را,نیابد بدو نیز اندیشه راه که او برتر از نام و ا...,9,13
...,...,...,...,...
49603,همش رای و هم دانش وهم نسب چراغ عجم آفتاب عرب,سرآمد کنون قصهٔ یزدگرد به ماه سفندار مد روز ارد,11,10
49604,سرآمد کنون قصهٔ یزدگرد به ماه سفندار مد روز ارد,ز هجرت شده پنج هشتادبار به نام جهانداور کردگار,10,9
49605,ز هجرت شده پنج هشتادبار به نام جهانداور کردگار,چواین نامور نامه آمد ببن ز من روی کشور شود پرسخن,9,11
49606,چواین نامور نامه آمد ببن ز من روی کشور شود پرسخن,از آن پس نمیرم که من زنده ام که تخم سخن من پرا...,11,14


In [11]:
input_min_max_len = total_data_df['input_poem_len'].min(), total_data_df['input_poem_len'].max()
output_min_max_len = total_data_df['output_poem_len'].min(), total_data_df['output_poem_len'].max()
print(f'Min: {np.min([input_min_max_len[0], output_min_max_len[0]])} \tMax: {np.max([input_min_max_len[1], output_min_max_len[1]])}')
MAX_POEM_LEN = np.max([input_min_max_len[1], output_min_max_len[1]])

Min: 5 	Max: 20


## Train and test split:

In [12]:
temp_df = pd.DataFrame({'input': total_data_df['input'], 'output': total_data_df['output']})
train, test = train_test_split(temp_df, test_size=0.15, random_state=1)
train = train.to_numpy()
test = test.to_numpy()

x_train, y_train = train[:, 0], train[:, 1]
x_test, y_test = test[:, 0], test[:, 1]

In [13]:
print(f'x_train: {x_train.shape}')
print(f'x_test: {x_test.shape}')

x_train: (42166,)
x_test: (7442,)


## A) Model training, LSTM

In [14]:
from transformers import BertConfig, BertTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers import get_linear_schedule_with_warmup

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torchsummary import summary

In [15]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'device: {device}')

train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

device: cuda:0
CUDA is available!  Training on GPU ...


In [16]:
TRAIN_BATCH_SIZE = 256
TEST_BATCH_SIZE = 256

MODEL_NAME_PATH = 'HooshvareLab/bert-fa-base-uncased'
OUTPUT_PATH = '/content/HW4/P1_model_LSTM.bin'

In [17]:
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME_PATH)

In [18]:
idx = np.random.randint(train.shape[0])
sample_poem = train[idx]
input_token = tokenizer.tokenize(sample_poem[0])
output_token = tokenizer.tokenize(sample_poem[1])
print(f"Original poem: {sample_poem}")
print(f"Input token: {tokenizer.convert_tokens_to_ids(input_token)}")
print(f"Input poem: {tokenizer.convert_tokens_to_string(input_token)}")
print(f"Output token: {tokenizer.convert_tokens_to_ids(output_token)}")
print(f"Output poem: {tokenizer.convert_tokens_to_string(output_token)}")

Original poem: ['دبیر بزرگ آن زمان لب ببست بانبوه اندیشه اندر نشست'
 'ازان پس چنین گفت بهرام را که هرکس جویا بود کامرا']
Input token: [4845, 3167, 2808, 3146, 4064, 14258, 2787, 3104, 92311, 7522, 11840, 4709]
Input poem: دبیر بزرگ ان زمان لب ببست بانبوه اندیشه اندر نشست
Output token: [13606, 2965, 3677, 3017, 9248, 2803, 2800, 14223, 19707, 2834, 3608, 2835]
Output poem: ازان پس چنین گفت بهرام را که هرکس جویا بود کامرا


In [19]:
class PoemDataset(torch.utils.data.Dataset):
  def __init__(self, tokenizer, poems, max_len=20):
    self.poems = poems
    self.tokenizer = tokenizer
    self.max_len = max_len
  
  def __len__(self):
    return len(self.poems)

  def __getitem__(self, item):
    poems = self.poems[item]
    poem_input = str(poems[0])
    poem_output = str(poems[1])

    encoding_input = self.tokenizer.encode_plus(
        poem_input,
        add_special_tokens=True,
        truncation=True,
        max_length=self.max_len,
        return_token_type_ids=True,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt'
    )
    encoding_output = self.tokenizer.encode_plus(
        poem_output,
        add_special_tokens=True,
        truncation=True,
        max_length=self.max_len,
        return_token_type_ids=True,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt'
    )

    inputs = {
        'poem_in': poem_input,
        'poem_out': poem_output,
        'input_ids': encoding_input['input_ids'].reshape(self.max_len),
        'output_ids':  encoding_output['input_ids'].reshape(self.max_len)
    }

    return inputs
  
def create_data_loader(x, tokenizer, max_len, batch_size):
  dataset = PoemDataset(poems=x,
                        tokenizer=tokenizer,
                        max_len=max_len)
  return torch.utils.data.DataLoader(dataset, batch_size=batch_size)

In [ ]:
train_dataloader = create_data_loader(train, tokenizer, MAX_POEM_LEN, TRAIN_BATCH_SIZE)
test_dataloader = create_data_loader(test, tokenizer, MAX_POEM_LEN, TEST_BATCH_SIZE)

In [ ]:
class EncoderLSTM(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, num_layers=n_layers, dropout=dropout)
        
        self.dropout = nn.Dropout(dropout)
    def forward(self, src):
        # src : [sen_len, batch_size]
        embedded = self.dropout(self.embedding(src))
        
        # embedded : [sen_len, batch_size, emb_dim]
        outputs, (hidden, cell) = self.rnn(embedded)
        # outputs = [sen_len, batch_size, hid_dim * n_directions]
        # hidden = [n_layers * n_direction, batch_size, hid_dim]
        # cell = [n_layers * n_direction, batch_size, hid_dim]
        return hidden, cell

class DecoderLSTM(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.emb_dim = emb_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, num_layers=self.n_layers, dropout=dropout)
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        
        # input = [batch_size]
        # hidden = [n_layers * n_dir, batch_size, hid_dim]
        # cell = [n_layers * n_dir, batch_size, hid_dim]
        
        input = input.unsqueeze(0)
        # input : [1, ,batch_size]
        
        embedded = self.dropout(self.embedding(input))
        # embedded = [1, batch_size, emb_dim]
        
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        # output = [seq_len, batch_size, hid_dim * n_dir]
        # hidden = [n_layers * n_dir, batch_size, hid_dim]
        # cell = [n_layers * n_dir, batch_size, hid_dim]
        
        # seq_len and n_dir will always be 1 in the decoder
        prediction = self.fc_out(output.squeeze(0))
        # prediction = [batch_size, output_dim]
        return prediction, hidden, cell

In [ ]:
class Seq2SeqLSTM(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            'hidden dimensions of encoder and decoder must be equal.'
        assert encoder.n_layers == decoder.n_layers, \
            'n_layers of encoder and decoder must be equal.'
        
    def forward(self, src, trg, teacher_forcing_ratio=1):
        # src = [sen_len, batch_size]
        # trg = [sen_len, batch_size]
        # teacher_forcing_ratio : the probability to use the teacher forcing.
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        # tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        # last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        
        # first input to the decoder is the <sos> token.
        input = trg[0, :]
        for t in range(1, trg_len):
            # insert input token embedding, previous hidden and previous cell states 
            # receive output tensor (predictions) and new hidden and cell states.
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            # replace predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            # decide if we are going to use teacher forcing or not.
            teacher_force = np.random.rand() < teacher_forcing_ratio
            
            # get the highest predicted token from our predictions.
            top1 = output.argmax(1)
            # update input : use ground_truth when teacher_force 
            input = trg[t] if teacher_force else top1
            
        return outputs

In [ ]:
# INPUT_DIM = 9620
# OUTPUT_DIM = 9620
INPUT_DIM = 100000
OUTPUT_DIM = 100000
ENC_EMB_DIM = 128
DEC_EMB_DIM = 128
HID_DIM = 128
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

encoder = EncoderLSTM(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
decoder = DecoderLSTM(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model_lstm = Seq2SeqLSTM(encoder, decoder, device).to(device)

In [24]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model_lstm.apply(init_weights)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model_lstm):,} trainable parameters')

The model has 39,028,384 trainable parameters


In [ ]:
optimizer = optim.Adam(model_lstm.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

In [28]:
def train(model, iterator, optimizer, criterion):
  model.train()
  epoch_loss = 0
  total_nums = 0
  corrects = 0
  
  for batch in tqdm(iterator, total=len(iterator), desc=f"Training..."):
    src = batch['input_ids'].to(device)
    trg = batch['output_ids'].to(device)
    total_nums += src.numel()

    optimizer.zero_grad()
    output = model(src, trg, 1)
    output_dim = output.shape[-1]

    output = output[1:].view(-1, output_dim)
    trg = trg[1:].view(-1)

    loss = criterion(output, trg)
    loss.backward()
    optimizer.step()

    epoch_loss += loss.item()
    corrects += torch.sum(torch.argmax(output, dim=1) == trg).item()

  return epoch_loss / len(iterator), corrects / total_nums*100

In [29]:
def evaluate(model, iterator, criterion):
  model.eval()
  epoch_loss = 0
  corrects = 0
  total_nums = 0
  with torch.no_grad():      
    for batch in tqdm(iterator, total=len(iterator), desc=f"Testing..."):
      src = batch['input_ids'].to(device)
      trg = batch['output_ids'].to(device)
      
      total_nums += src.numel()
                    
      output = model(src, trg, 0) # turn off teacher forcing.          
      output_dim = output.shape[-1]
      
      output = output[1:].view(-1, output_dim)
      trg = trg[1:].view(-1)
      loss = criterion(output, trg)      
      epoch_loss += loss.item()
      corrects += torch.sum(torch.argmax(output, dim=1) == trg).item()

  return epoch_loss / len(iterator), corrects / total_nums*100

In [ ]:
for epoch in range(10):
  print(f"At epoch {epoch}")
  train_loss, train_acc = train(model_lstm, train_dataloader, optimizer, criterion)
  test_loss, test_acc = evaluate(model_lstm, test_dataloader, criterion)
  print("Train loss {0:.3f}, train accuracy: {0:.3f}".format(train_loss, train_acc))
  print("Test loss {0:.3f}, test accuracy: {0:.3f}".format(test_loss, test_acc))

At epoch 0


Training...:   0%|          | 0/165 [00:00<?, ?it/s]

Testing...:   0%|          | 0/30 [00:00<?, ?it/s]

Train loss 6.135, train accuracy: 6.135
Test loss 5.361, test accuracy: 5.361
At epoch 1


Training...:   0%|          | 0/165 [00:00<?, ?it/s]

...

Train loss 4.612, train accuracy: 4.612
Test loss 4.946, test accuracy: 4.946
At epoch 7


Training...:   0%|          | 0/165 [00:00<?, ?it/s]

Testing...:   0%|          | 0/30 [00:00<?, ?it/s]

Train loss 4.578, train accuracy: 4.578
Test loss 4.858, test accuracy: 4.858
At epoch 8


Training...:   0%|          | 0/165 [00:00<?, ?it/s]

## B) GRU Model

In [20]:
class EncoderGRU(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim, hid_dim, num_layers=n_layers, dropout=dropout)
        
        self.dropout = nn.Dropout(dropout)
    def forward(self, src):
        # src : [sen_len, batch_size]
        embedded = self.dropout(self.embedding(src))

        # embedded : [sen_len, batch_size, emb_dim]
        outputs, hidden = self.rnn(embedded)
        # outputs = [sen_len, batch_size, hid_dim * n_directions]
        # hidden = [n_layers * n_direction, batch_size, hid_dim]
        # cell = [n_layers * n_direction, batch_size, hid_dim]
        return hidden

class DecoderGRU(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.emb_dim = emb_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim, hid_dim, num_layers=self.n_layers, dropout=dropout)
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden):
        
        # input = [batch_size]
        # hidden = [n_layers * n_dir, batch_size, hid_dim]
        # cell = [n_layers * n_dir, batch_size, hid_dim]
        
        input = input.unsqueeze(0)
        # input : [1, ,batch_size]
        
        embedded = self.dropout(self.embedding(input))
        # embedded = [1, batch_size, emb_dim]
        
        output, hidden = self.rnn(embedded, hidden)
        # output = [seq_len, batch_size, hid_dim * n_dir]
        # hidden = [n_layers * n_dir, batch_size, hid_dim]
        # cell = [n_layers * n_dir, batch_size, hid_dim]
        
        # seq_len and n_dir will always be 1 in the decoder
        prediction = self.fc_out(output.squeeze(0))
        # prediction = [batch_size, output_dim]
        return prediction, hidden

In [21]:
class Seq2SeqGRU(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            'hidden dimensions of encoder and decoder must be equal.'
        assert encoder.n_layers == decoder.n_layers, \
            'n_layers of encoder and decoder must be equal.'
        
    def forward(self, src, trg, teacher_forcing_ratio=1):
        # src = [sen_len, batch_size]
        # trg = [sen_len, batch_size]
        # teacher_forcing_ratio : the probability to use the teacher forcing.
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        # tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        # last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden = self.encoder(src)
        
        # first input to the decoder is the <sos> token.
        input = trg[0, :]
        for t in range(1, trg_len):
            # insert input token embedding, previous hidden and previous cell states 
            # receive output tensor (predictions) and new hidden and cell states.
            output, hidden = self.decoder(input, hidden)
            
            # replace predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            # decide if we are going to use teacher forcing or not.
            teacher_force = np.random.rand() < teacher_forcing_ratio
            
            # get the highest predicted token from our predictions.
            top1 = output.argmax(1)
            # update input : use ground_truth when teacher_force 
            input = trg[t] if teacher_force else top1
            
        return outputs

In [22]:
train_dataloader = create_data_loader(train, tokenizer, MAX_POEM_LEN, 8)
test_dataloader = create_data_loader(test, tokenizer, MAX_POEM_LEN, 8)

In [25]:
# INPUT_DIM = 9620
# OUTPUT_DIM = 9620
INPUT_DIM = 100000
OUTPUT_DIM = 100000
ENC_EMB_DIM = 64
DEC_EMB_DIM = 64
HID_DIM = 4
N_LAYERS = 1
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

encoder = EncoderGRU(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
decoder = DecoderGRU(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model_gru = Seq2SeqGRU(encoder, decoder, device).to(device)
model_gru.apply(init_weights)

Seq2SeqGRU(
  (encoder): EncoderGRU(
    (embedding): Embedding(100000, 64)
    (rnn): GRU(64, 4, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): DecoderGRU(
    (embedding): Embedding(100000, 64)
    (rnn): GRU(64, 4, dropout=0.5)
    (fc_out): Linear(in_features=4, out_features=100000, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [26]:
optimizer = optim.Adam(model_gru.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

In [31]:
for epoch in range(5):
  print(f"At epoch {epoch}")
  train_loss, train_acc = train(model_gru, train_dataloader, optimizer, criterion)
  test_loss, test_acc = evaluate(model_gru, test_dataloader, criterion)
  print("Train loss {0:.3f}, train accuracy: {0:.3f}".format(train_loss, train_acc))
  print("Test loss {0:.3f}, test accuracy: {0:.3f}".format(test_loss, test_acc))

At epoch 0


Training...:   0%|          | 0/5271 [00:00<?, ?it/s]

Testing...:   0%|          | 0/931 [00:00<?, ?it/s]

Train loss 4.778, train accuracy: 4.778
Test loss 4.686, test accuracy: 4.686
At epoch 1


Training...:   0%|          | 0/5271 [00:00<?, ?it/s]

...

Testing...:   0%|          | 0/931 [00:00<?, ?it/s]

Train loss 4.613, train accuracy: 4.613
Test loss 4.648, test accuracy: 4.648
At epoch 4


Training...:   0%|          | 0/5271 [00:00<?, ?it/s]

Testing...:   0%|          | 0/931 [00:00<?, ?it/s]

Train loss 4.589, train accuracy: 4.589
Test loss 4.633, test accuracy: 4.633
